# BGD的非凸优化问题2：网络深度带来的问题

<font color=norange>**model框架：**
$$
input \rightarrow \underset{optimization}{{\underbrace{\overset{f(x,w)}{\overbrace{feature\ representation + classifier}} \rightarrow \overset{Div,reg}{ \overbrace{ cost function}}}}} 
$$</font>
**整个model可以拆分为两大类问题：模型结构设计和优化求解。梯度下降是求解优化问题的标准方法，但实践中有两类典型问题要解决**：
1. loss surface形态不好带来的各种问题
2. 网络深度带来的计算问题

## I. forward propagation

### I.1 FP过程中的信息流
1. 数学公式
$$
\begin{align}
z^{(1)}&=f(s^{(0)})=f(z^{(0)}@w^{(0)}) \\
z^{(2)}&=f(s^{(1)})=f(z^{(1)}@w^{(1)})=f(f(w^{(0)})@w^{(1)})\\
z^{(3)}&=f(s^{(2)})=f(z^{(2)}@w^{(2)}) \\
&=f(f(z^{(1)}@w^{(1)})@w^{(2)})=f(f(f(z^{(0)}@w^{(0)})@w^{(1)})@w^{(2)})\\
...\\
z^{(h)}&=f(s^{(h-1)})=f(z^{(h-1)}@w^{(h-1)})\\
&=f\left(...f\left (f(f(z^{(0)}@w^{(0)})@w^{(1)})@w^{(2)}\right )...@w^{(h-1)}\right ) \\
\end{align}
$$
2. dimension变化
$$
\begin{align}
&\ \  z^{(0)}(N,D_0),\ \ z^{(1)}(N, D_1),\ \  ... ,z^{(h-1)}(N, D_{h-1}),z^{(h)}(N, D_h),..., z^{(H)}(N, D_{h}) \\
& w^{(0)}(D_0, D_1), w^{(1)}(D_1, D_2), ..., w^{(h-1)}(D_{h-1}, D_h),\ \ \  ...\ \ \  ...,\ \ \  w^{(H)}(D_h, D_{out})
\end{align}
$$

3. 样本信息传递：以分类问题为例
 - <font color=deeppink>**从矩阵形态可见，每层$z$都有N行，每一行对应一个样本的信息传递。**</font>
 - 第一部分：计算score
   <img src="pics/foreward.png" alt="alt text" width="560"/>

 - 第二部分：从score到Loss
$$
\begin{align}
score=\begin{pmatrix}
 - & score_1 & -\\
 - & score_2 & -\\
  & ... & \\
 - & score_N &-
\end{pmatrix}_{N\times C}\rightarrow l=\begin{pmatrix}
l_1\\
l_2\\
...\\
l_N
\end{pmatrix}_{N\times 1}\rightarrow Loss=\frac{1}{N}\sum_{i=1}^{N} l_i + reg
\end{align}
$$

4. 每一层中的运算
 - $z$每进入新的一层就做两个运算:矩阵乘法$z@w$和non-linearity
 - <font color=orange>矩阵乘法$z@w$中，如果将$w$用SVD转换成$U\Sigma V^T$，那么$z@w$相当于对z做旋转、伸缩降维或升维、再旋转。</font>
 - non-linearity的三种主要函数：(简化以三维空间做分析)
   - Relu把所有点挤压到第一象限的空间中
   - tanh把所有点投影到每个轴上(-1,1)的正方体空间
   - Sigmoid把所有点投影到每个轴上(0,1)的正方体空间

<font color=red>[**rk's延伸思考**]把w从SVD的角度来看，给batchnorm操作提供了合理性。参考pca中，每个batch都要先中心化才好得到结果。</font>

### I.2 Forward propagation中的问题: covariate shift

1. 理想状态
- **假设1**：$w_i$的值给定。那么只要最初的输入$z_0=x$的分布不变，则线性变化$w_0·x$的分布也稳定，作为下一层输入的$z_1=f(w_0·x)$的分布也稳定。同理，后续每一层的分布也稳定。
- **放松为假设2**：$w_i$的值在每次迭代时都发生变化，但是能保证线性变化$w_0·x$的分布稳定。此时，作为下一层输入的$z_1=f(w_0·x)$的分布也稳定。同理，后续每一层的分布也稳定。

2. **实际问题**：\
(1) 每个batch得到的数据与其他batch数据的分布都很难保证一致。假设$w$不变，每层拿到的数据都因为最初的x batch变化而变化。\
(2) 更严重的是，迭代会导致$w$的值持续变化，加大$w_0·x$的分布变化，后续每一层的输入$z_{i+1}=f(w_{i}·z_{i})$分布也都变化。
- 简化以单层网络为例来说明，如果input的分布总是变化，也就相当于每次输入对应的loss surface都在变化，$w$很难收敛。这比mini-batch SGD的情况还糟糕，因为mini-batch中的分布是一样的，只是因为抽样导致了样本特征变化。
3. **solution:batch norm** \
batchnorm给出的处理方式是锁定input的分布。相当于一种Regularization，各层w的取值保持各自分布特征(均值和方差)不变。

## II. Backward propagation
### II.1 BP过程中的信息流
#### II.1.1 梯度公式
$$
\begin{align}
ds^{(h)} & = dz^{(h+1)}*\nabla f^{(h)} \\
& = ds^{(h+1)}@\left(w^{(h+1)}\right)^T*\nabla f^{(h)}\\
& = ds^{(H)}@\left(w^{(H)}\right)^T*\nabla f^{(H-1)}...@\left(w^{(h+2)}\right)^T*\nabla f^{(h+1)}@\left(w^{(h+1)}\right)^T*\nabla f^{(h)}\\
& = \underset{N,D_H\times D_H,D_{H-1}}{\underbrace{dscore@\left(w^{(H)}\right)^T}}  *\underset{N,D_{H-1}}{\underbrace{\nabla f^{(H-1)}}}  ...@\underset{D_{h+2},D_{h+1}}{\underbrace{\left(w^{(h+2)}\right)^T}}  *\underset{N,D_{h+1}}{\underbrace{\nabla f^{(h+1)}}}  @\underset{D_{h+1},D_{h}}{\underbrace{\left(w^{(h+1)}\right)^T}}  *\underset{N,D_{h}}{\underbrace{\nabla f^{(h)}}}
\end{align}
$$
$$
\begin{align}
dw^{(h)}&=\left(z^{(h)}\right)^T@ds^{(h)} \\
&=\left(z^{(h)}\right)^T@\color{Red}{dscore@\left(w^{(H)}\right)^T*\nabla f^{(H-1)}...*\nabla f^{(h+1)}@\left(w^{(h+1)}\right)^T*\nabla f^{(h)}}
\end{align} 
 $$

#### II.1.2 信息流示意图
   - <font color=green>合理简化，假设$L_i$的计算只使用$score_i$的信息，不使用其他样本的score信息</font>
   - 第一部分：从Loss到score。
   $$
\begin{align}
dscore=\frac{\partial Loss}{\partial score}=\frac{1}{N}\begin{pmatrix}
 - & \frac{\partial l_1}{\partial score_1} & -\\
 - & \frac{\partial l_2}{\partial score_2} & -\\
  & ... & \\
 - & \frac{\partial l_N}{\partial score_N} &-
\end{pmatrix}_{N\times C}\leftarrow \frac{\partial Loss}{\partial l}=\frac{1}{N} \begin{pmatrix}
1\\
1\\
...\\
1
\end{pmatrix}_{N\times 1}
\end{align}
$$
   - 第二部分：从score反馈到整个网络
     <img src="pics/backprop.png" alt="alt text" width="560"/>

#### II.1.3 样本信息传递方式
1. <font color=deeppink>**持续向前(浅层)传递的是dz和ds。上图可见，每层的ds和dz都有N行，每一行传递一个样本从loss中取得的信息。**</font>
   - <font color=norange>$dz^{(h)}$中每行对应一个样本，第i行是第i个样本的$ds^{(h)}$被$(w^{(h)})^T$重新投影到$D_{in}$维的空间的结果。</font>
     - 公式：$$\begin{align}
     dz^{(h)}&=ds^{(h)}@(w^{(h)})^T \\
&=\begin{pmatrix}
 - & ds_1 & -\\
 - & ds_2 &- \\
 - & ... & -\\
 - & ds_N &-
\end{pmatrix}_{(N\times D_{out})}@\begin{pmatrix}
 | & | & &| \\
 w_1^T & w_2^T& ... &  w_N^T\\
 | & | &  &|
\end{pmatrix}_{(D_{out}\times D_{in})}
\end{align}$$
    - <font color=norange>$ds^{(h)}= dz^{(h+1)}*\nabla f^{(h)}$,每层ds都是下一层(更深一层)dz被$\nabla f$过滤后的结果。</font>

2. **dw所接收的信息是batch中所有N个样本提供的梯度信息的均值。**
$$dw=z^T@ds$$
   - N=1时,dw是rank为1的矩阵，它是单个样本中得到的梯度信息ds与本层输出z做矩阵乘法得到的结果。
$$
\begin{align}
dw&=\begin{pmatrix}
| \\
z^T \\
|
\end{pmatrix}_{(D_{in}\times 1)}@\begin{pmatrix}
 - & ds &-
\end{pmatrix}_{(1\times D_{out})}\\
\end{align}
$$
   - N > 1时，dw是N个rank为1的矩阵的均值。每个rank为1的矩阵$z_i^T@ds_i$都是第i个样本得到的梯度信息$ds_i$与该样本在本层的输出$z_i$做矩阵乘法。
   $$
\begin{align}
dw&=\frac{1}{N}*\begin{pmatrix}
 | & | & &| \\
 z_1^T & z_2^T& ... &  z_N^T\\
 | & | &  &|
\end{pmatrix}_{(D_{in}\times N)}@\begin{pmatrix}
 - & ds_1 & -\\
 - & ds_2 &- \\
 - & ... & -\\
 - & ds_N &-
\end{pmatrix}_{(N\times D_{out})}\\
&=\frac{1}{N}\sum_{i=1}^{N}z_i^T@ds_i \\
&=\frac{1}{N}\sum_{i=1}^{N}z_i^T@(\nabla f_i*dz_i^{'})\\
\end{align}
$$<font color=blue>注：这里$\frac{1}{N}$是从ds中抽出来的，为了说明这里有均值的含义。</font>

### II.2 深度网络中引起梯度消失和梯度爆炸的三个因素

1. 从梯度公式看三个影响因素
$$
\begin{align}
dw^{(h)} & = \left(z^{(h)}\right)^T@ds^{(h)}= \left[f(s^{(h-1)})\right]^T@ds^{(h)} \\
ds^{(h)}&=dscore@\left(w^{(H)}\right)^T*\nabla f^{(H-1)}...@\left(w^{(h+2)}\right)^T*\nabla f^{(h+1)}@\left(w^{(h+1)}\right)^T*\nabla f^{(h)}\\
&...[1] \\
\end{align}
$$
 - <font color=red>[1]式计算必须是从左到右进行。</font> 
 - 从上面公式可知，dw由f和ds决定，每层$ds$由更深层的$w$和$\nabla f$决定。所以，影响dw的三个直接因素分别是$\nabla f, f和w$的取值。

#### 第1种导致梯度消失的因素： $\nabla f=0$导致$ds=0$从而使$dw=0$
1. **单个$\nabla f$的过滤作用**：
   - Relu，$\nabla f$取0和1，凡是0的位置，$dz$信息被拦截;其他位置才能通过
   - Sigmoid，在|s|>5的位置$\nabla f\rightarrow0$,$dz$信息被拦截，其他位置$\nabla f\in (0, 0.25)$,梯度信息被压缩
   - tanh，在|s|>3的位置$\nabla f\rightarrow0$,$dz$信息被拦截，其他位置$\nabla f\in (0, 1]$,梯度信息被压缩
   <img src="pics/sigmoid.png" style="width:40%; display:inline-block; margin-right:10px;"> <img src="pics/tanh.png" style="width:40%; display:inline-block;">

2. **多层$\nabla f$持续压缩$ds$中的元素**
   - 根据[1]式，如果将一次inner product和一次element-wise product视为一组，第h层的ds的计算就是做H-h次该组合运算。
   - $\nabla f$的乘法运算会将其左边矩阵的元素值过滤和缩小，如果下一次右边的矩阵运算中$W$不能起到反向放大的作用，多次运算后，ds中的元素就会越来越多趋近于0。<font color=green>[W作用的说明详见后文梯度爆炸部分]</font>
   - <font color=deeppink>**这也意味着，网络越深，梯度消失会越严重。**</font>

#### 第2种导致梯度消失的因素：$f=0$直接使$dw=0$
$$dw^{(h)} = \left(z^{(h)}\right)^T@ds^{(h)}= \left[f(s^{(h-1)})\right]^T@ds^{(h)}$$
- 如果$f$中元素值为0，也会导致$dw$收到的梯度信息减弱甚至消失。
- <font color=deeppink>**而常见的activation function都有一个问题是，如果w的初始值大小不合适，那么网络层次越深，f中的元素会越来越多的取0值。**</font>

#### 第3种导致梯度消失和梯度爆炸的因素：$w$影响$ds$从而影响$dw$
 - 这种情况主要发生在RNN中，因为<font color=norange>在RNN中，$w^{(i)}=w^{(i)}=w$，此时梯度公式为：
$$
ds^{(h)} = dscore@w^T*\nabla f^{(H-1)}@w^T*\nabla f^{(H-2)}...@w^T*\nabla f^{(h+1)}@w^T*\nabla f^{(h)} 
$$</font>

 - 根据矩阵乘法的特点，$wx$在$w$的singular value小于1的方向上shrink x，此时梯度消失情况会被加重。在$w$的singular value大于1的方向上expand x，此时可能导致gradient explode。

### II.3 梯度消失和信息传递的关系

#### II.3.1 单层$\nabla f=0$拦截信息，导致的梯度消失
<font color=red>注：这里强调单层是为了区别于另一种情况：多层$\nabla f<1$叠加带来的梯度严重衰减甚至消失。并不是说只影响一层的梯度。</font>
#### <font color=blue>$\nabla f$影响$dw$的方式</font>
$\nabla f$通过影响$ds$从而影响$dw$。由于$ds$会从深层向浅层传递，所以$\nabla f$不仅会影响本层$dw$，还会影响所有浅层的$dw$。
   - 对本层dw的影响：
     $$
\begin{align}
dw&=\frac{1}{N}\sum_{i=1}^{N}z_i^T@ds_i \\
&=\frac{1}{N}\sum_{i=1}^{N}z_i^T@(\nabla f_i*dz_i^{'}) \\
&=\frac{1}{N}\sum_{i=1}^N\begin{pmatrix}
| & | & & |\\
\nabla f_{i1}dz_{i1}^{'}z_i^T & \nabla f_{i2}dz^{'}_{i2}z_i^T  & ... & \nabla f_{iD^{'}}dz^{'}_{iD^{'}}z_i^T\\
| & |& & |
\end{pmatrix}_{(D_{in}\times D_{out})}
\end{align}
$$
   - 向前传递的信息：
$$dz^{(h)}=ds^{(h)}@(w^{(h)})^T$$

#### <font color=blue>$\nabla f$不同范围取0值对dw的影响</font>
1. **如果$\nabla f$中单个元素为0**
     - 比如：$\nabla f_{ij}=0$，对应第i个样本的$\nabla f_i$中第j个元素值为0。由上式可知，此时由第i个样本提供的梯度信息矩阵$z_i^T@(\nabla f_i*dz_i^{'})$中，第j列全0。
     - 对本层w迭代的影响：
       - 这意味着，第i个样本对w的第j列元素的更新不提供有效信息。
       - 除非单样本SGD的情况，不然此时w仍然可以利用其他样本提供的信息继续迭代。
     - 对浅层w迭代的影响：
      - 无直接影响
3. **如果$\nabla f$中有整行元素为0**
     - 比如：$\nabla f_{i}=0$，对应第i个样本的$ds_i=0$。
     - 对本层w迭代的影响：
       - 这意味着，第i个样本对本层w中所有元素的更新都不提供有效信息。
       - 除非单样本SGD的情况，不然此时本层w仍然可以利用其他样本提供的信息继续迭代。
     - 对浅层w迭代的影响：
       - $ds^{(h)}$整行为0会向前传递给$dz^{(h)}$,然后传给$ds^{(h-1)}$，所有浅层ds中对应行的元素都是0。所以，第i个样本对该层之前所有w的迭代都不再提供有效信息。
       - 除非单样本SGD的情况，不然此时所有浅层w仍然可以利用其他样本提供的信息继续迭代。
4. **如果有某层整层$\nabla f=0$**
     - 对本层w迭代的影响：
       - 此时该层ds=0，所有样本都不提供信息，dw=0，该层w停止迭代。
     - 对浅层w迭代的影响：
       - 由于ds向前传递，比它浅层的所有ds都→0，因此对应的dw也全为0，整个模型停止迭代。
5. 即使没有发生整层$\nabla f=0$,也可能导致$ds=0$，模型停止迭代。
     - 假如一个batch中有k个样本，他们分别在不同的层上发生了整行$\nabla f=0$。位于这些层前面的浅层的w会停止迭代，从而使模型停止迭代。
5. **如果$\nabla f$中有整列元素为0**
     - 比如：$\nabla f_{·j}=0$，对应每个样本的第j个元素值都为0。此时$ds_{·j}=0$，所有样本提供的梯度信息矩阵的第j列都是0，dw的第j列全0。
     - 对本层w迭代的影响：
       - w的第j列元素停止迭代。
     - 对浅层w迭代的影响：
       - 0值不会直接向前传递。

#### II.3.2  $f=0$导致的梯度消失
#### <font color=blue>$f$影响$dw$的方式</font>
$f=0$主要影响本层dw的取值。只有满足一定条件，$f=0$才能通过foreward propagation使深层的$f$取值也为0而间接影响深层的$dw$。
<font color=norange>具体条件是：$f(s)=0时有f(f(s)@w)=0也成立$。</font>
 - 常见的activation function中：
   - <font color=red>ReLU满足该条件，这也是Dead ReLU发生的原因</font>
   - sigmoid不满足该条件
   - tanh仅在s正好取0值时满足，影响不大

#### <font color=blue>$f$不同范围取0值对dw的影响</font>
$$
\begin{align}
dw&=\frac{1}{N}\sum_{i=1}^{N}z_i^T@ds_i \\
&=\frac{1}{N}*\sum_{i=1}^{N}\begin{pmatrix}
| \\
z_i^T \\
|
\end{pmatrix}_{(D_{in}\times 1)}@\begin{pmatrix}
 - & ds_i &-
\end{pmatrix}_{(1\times D_{out})}
\end{align}
$$

1. $f$中单个元素为0
   - 假设$f$第i行中第j个元素值为0，相当于$z^T$中第i列第j个元素$z^T_{ij}=0$。此时，一维矩阵$z_i^T@ds_i$的第j行全0。
   - 对本层dw的影响：
     - 意味着第i个样本对w第j行的更新不提供信息。
     - 除非单样本SGD的情况，否则此时w仍然可以利用其他样本提供的信息继续迭代。

2. $f$中整行值为0
   - 假设$f_i=0$，即$z^T_i=0$，此时一维矩阵$z_i^T@ds_i$值全为0。
   - 对本层dw的影响：
     - 意味着第i个样本对w的更新不提供信息。
     - 除非单样本SGD的情况，否则此时w仍然可以利用其他样本提供的信息继续迭代。

3. $f$中整列值为0
   - 假设$f_j=0$，即$z^T$第j行为0，此时dw中第j行值全0。
   - 对本层dw的影响：
     - 意味着所有样本都不提供w第j行元素更新的信息，这些元素的迭代停止。

4. $f=0$对更深层的影响（前向传递）：**Dead ReLU**
$$
\begin{align}
ReLU\begin{cases}
  & \text{ if } s>0,  f(s)=s, \nabla f=1 \\
  & \text{ if } s\le 0, f(s)=0, \nabla f=0
\end{cases}
\end{align}
$$
     - 如果$h$层第i行整行$f=0$，从该层开始，第i个样本对所有w的更新都不提供信息。
       - 分析：如果整行$f(s)=0$，forward运算中，$s^{h+1}=f^{h}@w^{h}$的对应行元素值也是0。$f^{(h+1)}(s^{(h+1)})$对应行的取值还是0。同理，$h$层之后所有层的$f$对应行的值全0。第i个样本对所有深层网络的w更新不提供信息。<font color=orange>[注，这一条tanh也符合]</font>
       - 此时该层对应行的$\nabla f(s)=0$，第i个样本对所有浅层网络的w迭代不提供信息。
     - <font color=norange>ReLU之所以严重，不只是因为一旦$f$有整行为零，那么对应行的样本无效。最大的问题是，ReLU的运算本身会导致大量出现$f=0$的情况。</font>
       - 网络深度越深，$f$中元素取0值的越来越多，6层之后，0值超过一半。\
       <font color=green>[详见笔记：opt_code_gradient_disappear]</font>

### II.4 解决方式
根据梯度消失发生的原因，有两个方向的思路：
1. 减少$f=0$和$\nabla f=0$导致的梯度直接消失
2. 网络层度较深时，要减少多层$\nabla f < 1$带来的对梯度的持续压缩
第二个问题相对好解决，因为ReLU没有这个问题，tanh的问题没有sigmoid严重。所以一个简单的解决方式是不用sigmoid，少用tanh。
3. 找到更好的网络结构

#### II.4.1 影响$f$和$\nabla f$取值的因素
1. s的取值导致的$f=0$和$\nabla f=0$
   - tanh中，当$|s|>3$时，$\nabla f=0$。
   - sigmoid中，当$s<-5$时，$f=0$和$\nabla f=0$同时发生；当$s>5$时，$\nabla f=0$。
   - ReLU中，$s\le 0$时，$f=0$和$\nabla f=0$同时发生。

$$
ReLU\begin{align}
\left\{\begin{matrix}
s\le 0时， &  \color{Red} {f = 0}, & \color{Red} {\nabla f = 0}  \\
s>0时，& f = s, & \nabla f = 1
\end{matrix}\right.
\end{align}
$$

$$
\begin{align}
tanh
\left\{\begin{matrix}
s<-3时， & f\rightarrow -1, & \color{Red} {\nabla f\rightarrow 0}  \\
-3\le s\le 3时， & f\in (-1, 1), & \nabla f \in (0,1) \\
s>3时，& f\rightarrow 1,& \color{Red} {\nabla f\rightarrow 0}
\end{matrix}\right.
\end{align}$$

$$
sigmoid\begin{align}
\left\{\begin{matrix}
s<-5时， & \color{Red} {f\rightarrow 0}, & \color{Red} {\nabla f\rightarrow 0 }\\
-5\le s\le 5时， & f\in (0, 1), & \nabla f \in (0,0.25) \\
s>5时，& f\rightarrow 1,& \color{Red} {\nabla f\rightarrow 0 }
\end{matrix}\right.
\end{align}
$$

2. 决定s取值的因素是z和w，也即$f和w$
$$
\begin{align}
s & = z@w\\ 
z & =f(s^{-}) \ \ ,这里s^{-}表示s^{(h-1)} \\
单个元素：\\
s_{ij} & = z_i@w_j\\
z_{ij} & =f(s^{-}_{i})@w_j=f(z_i^-@w^-)@w_j
\end{align}
$$
<font color=blue>**可见，主要的可调节因素是：activation function的选择和w的取值范围控制。**</font>

#### II.4.2 典型activation function的计算特征分析

1. **sigmoid**
     - $f$：$s<-5$时，$f$趋近于0。
     - $\nabla f$：
       - $\nabla f$：$|s|>5$时，$\nabla f \rightarrow 0$。所以，$|w|$较大时，很多位置梯度趋近于0.
       - <font color=orange>sigmoid更严重的问题是，$\nabla f$的最大值只有0.25，在深度网络中，对梯度的缩减效应太强。</font>
         - **solution**: <font color=brown>**所以深度网络中很少用sigmoid做activation function。**</font>

2. **tanh**
     - $f$：虽然仅在s刚好为0时取0，但在深度网络中，随着网络层数加深，$f$取值也会越来越集中在靠近0的位置。
       1.  因为<font color=deeppink>$|tanh(x)|<|x|$</font>，如果$tanh(s)@w$不能通过w放大的话，下层运算$tanh(tanh(s)@w)$会进一步压缩矩阵元素值的大小。在forward propagation中，网络越深的位置，越多矩阵的元素值会趋近于0。
       2. 实际上w还可能加重上面的问题。$x@w$运算中，在w特征值>1的方向上会扩张x，在w特征值<1的方向上会压缩x。这里在w特征值<1的方向上，情况会更差。
       3.  而且w取值不能太大，否则即使可以减轻tanh函数的压缩作用，但会把tanh取值推到-1和1上，这时候又会带来$\nabla f=0$的问题。
     - <font color=green>**solution**: </font>
         - <font color=green>**Batchnorm会减轻这个问题**</font>
         - <font color=blue>Xavier提出了解决上面第2项问题的办法，方法是优化w的初始值，通过满足$Var(z@w)=Var(z)$让w在FP中不加重维度压缩。</font>
     - $\nabla f$：
       - $|s|>3$时，$\nabla f \rightarrow 0$。所以，$|w|$大时，很多位置梯度趋近于0.
         - **solution**: <font color=norange>由于tanh的取值范围在(0,1)，$|w|$的合理范围应该不难找到，这个范围要考虑s的维度数。</font>
       - <font color=orange>即使$\nabla f>0$，因为tanh的$\nabla f$取值范围是(0,1]，多层叠加后，对梯度有较强的缩减作用。</font>
         - **solution**: <font color=norange>深度网络中如果用tanh，就要找到对抗这种压缩的结构或方法。</font>


3. **ReLU**
     - $f$：所有小于0的输入，ReLU的输出都是0。所以在w均值为0(有正有负)的情况下，在深度网络中，随着网络层数加深，$f(s)$取值中越来越多的元素值为0。
       1. 这里$f$被压缩到0的动因是ReLU将所有小于0的s都投影到0上。和tanh相比，ReLU不会压缩s>0的部分。
       2. 和tanh相比，w在这里对$f=0$的趋势没有系统性的影响。
     - **solution**: 
         - <font color=norange>**Batchnorm会减轻这个问题**</font>
       - Xavier也给ReLU提供了w的初始化方式。<font color=red>[rk]但是从这里与tanh的对比可知，Xavier对ReLU的效果应该不如tanh。这点可以通过笔记opt_code_gradient_disappear部分的结果验证</font>
     - $\nabla f$：
       - $f$值为0的位置，$\nabla f$也为0。所以网络一旦加深，梯度消失严重。
         - **solution**: <font color=green>如果能解决$f=0$的问题，这里也会同时被解决。</font>
       - ReLU与tanh和sigmoid不同，不存在叠加压缩的问题。

#### II.4.2 小结
1. 选择性质更合适的activation：tanh和sigmoid都有这个问题，tanh没有sigmoid严重，所以RNN用tanh，不用sigmoid。CNN用Relu
2. 让f(z)不趋于相同值，即$Var(f(z))≠0$：Xaiver，MSRA(Kaiming)
   - 通过weight initialization可以在刚开始迭代的时候近似满足$Var(z_{i+1}))=Var(z_{i})$，缓解上述问题。ResNet中是$Var(f(w_{i}x_{i})+x)=Var(x_{i})$
   - 但一旦开始迭代后，$w$的scale取决于learning rate和梯度的大小，仅靠初始值的上述安排无法保证迭代过程中$Var(z_{i+1}))=Var(z_{i})$继续成立。
3. 在初始化的基础上，为了控制迭代过程中$w$的scale在合理区间，不变得太大，发生saturate，只能用small learning rates，代价是降低了converge rate。
   - 但总得来说，**以上方法都没有从本质上解决问题。**

**一种更强的约束是Batchnorm**
 - 让每一层的input的分布保持稳定$distribution(z^{[t+1]}_i)=distribution(z^{[t]}_i)$，而不仅是让每一层input的方差保持层间的一致性$Var(z_{i+1}))=Var(z_{i})$。前者和initialization方法一样可以让$Var(z)≠0$，在此基础上，附加了更多约束。并且还有一个优点是，initialization的各种方法难以在迭代过程中持续保证$Var(z_{i+1}))=Var(z_{i})$。而Batchnorm则可以做到。
 - 图示：MLP中不同initial weight scale下，batchnorm效果
<img src="pics/batchnormvsscale.png" alt="alt text" width="420"/>

### II.5 BP中，如果所有位置$f\ge0$，单样本SGD时w迭代会呈zigzag形
1. 问题说明：<font color=red>**sigmoid和relu的单样本SGD model中**</font>，由于每次只有一个样本，此时$w$中同一列元素迭代时，由梯度决定的迭代方向限制在全+和全-空间，发生zigzag，降低迭代速度。
 - 分析：
$$
\begin{align}
dw & = z^T@\nabla f*dz^{'}\\
& =\begin{pmatrix}
| \\
z^T \\
|
\end{pmatrix}_{(D_{in}\times 1)}@\begin{pmatrix}
 - & \nabla f*dz^{'} &-
\end{pmatrix}_{(1\times D_{out})}\\
& =\begin{pmatrix}
| \\
z^T \\
|
\end{pmatrix}_{(D_{in}\times 1)}@\begin{pmatrix}
\nabla f_1dz^{'}_1 & \nabla f_2dz^{'}_2 & ... & \nabla f_{D^{'}}dz^{'}_{D^{'}}
\end{pmatrix}_{1\times D^{'}} \\
&=\begin{pmatrix}
| & | & & |\\
\nabla f_1dz^{'}_1z^T & \nabla f_2dz^{'}_2z^T  & ... & \nabla f_{D^{'}}dz^{'}_{D^{'}}z^T\\
| & |& & |
\end{pmatrix}_{(D_{in}\times D_{out})}
\end{align}
$$
**结论：<font color=norange>dw的shape是$(D,D^{'})$，其中每一列的符号由其对应的scalar：$dz_j^{'}$决定，每次迭代的方向都只能同+或同-。</font>**

2. <font color=green>mini-batch和Batch GD这个问题不大。因为多个样本有正有负互相抵消。</font>
 - 分析：假设每个batch有b个样本，和上面分析类似，有：
 $$
\begin{align}
dw & = z^T@\nabla f*dz^{'}\\
& = \left(z^T\right )_{D\times b}@\begin{pmatrix}
\nabla f_1dz^{'}_1 & \nabla f_2dz^{'}_2 & ... & \nabla f_{D^{'}}dz^{'}_{D^{'}}
\end{pmatrix}_{b\times D^{'}} \\
dw中的第j列：\\
dw_{·j}& = \left(z^T\right )_{D\times b}@\left (\nabla f_jdz^{'}_j\right )_{b\times 1}
\end{align}
 $$
 此时，$dz_j^{'}$是vector，它包括的元素有正有负，所以dw中每一列的元素也有正有负

3. <font color=green>tanh没有这个问题，因为tanh函数的值有正有负</font>
4. 把上面的分析一般化，不仅是全正全负的activation out不利于求解，均值不为0都有类似问题。 \
<font color=red>**这也给preprocess data提供了一个zero-center操作的理由。因为x作为输入就是这里的$z_0$**</font>